In [1]:
# Importing file
import pandas as pd
import numpy as np
import warnings
warnings.filterwarnings('ignore')

In [2]:
def is_number(n):
    if str(n).lower() == 'nan':
        return False
    try:
        float(n)
    except ValueError:
        return False
    return True

In [3]:
all_data = pd.read_excel("19965 - 19997_Dist 3_Contractor WB_DateMod_20180926 102843.xlsx", sheetname='Audit')
all_data.iloc[:,3] = all_data.iloc[:,3].astype(str)
all_data.iloc[:,1] = all_data.iloc[:, 1].astype(str)
# Removing the row that doens't start with $
all_data = all_data[all_data.iloc[:, 0].str.contains("^\$")]
all_data = all_data[all_data.iloc[:, 3].str.contains("^QC.*")]
all_data = all_data[all_data.iloc[:, 1].apply(lambda x: is_number(x))]
all_data["hash"] = all_data.iloc[:, 0].str.cat(all_data.iloc[:, 3])
all_data['hash'] = all_data["hash"].apply(lambda x: hash(x))
all_data.head()

,Unnamed: 0,Unnamed: 1,Unnamed: 2,Unnamed: 3,Unnamed: 4,Unnamed: 5,Unnamed: 6,Unnamed: 7,Unnamed: 8,Unnamed: 9,Unnamed: 10,Unnamed: 11,Unnamed: 12,Unnamed: 13,Unnamed: 14,hash
215,$U$20,22275,2018-08-13 03:38:17,QC(1),C:\Users\aaron.mabee\Dropbox\2018 Jobs and Pro...,Kuna to Meridian\Production\Contractor,QASP v1.1 workbook_2018-08-06_Contractor.xlsb,MTN.CRHOMG.COM,HL9YYZ1,\\MTNDCUT01,aaron.mabee,Oldcastle,19965 - 19997,NaN,NaN,1833723957341059065
219,$BG$23,1,2018-08-13 03:38:40,QC(1),C:\Users\aaron.mabee\Dropbox\2018 Jobs and Pro...,Kuna to Meridian\Production\Contractor,QASP v1.1 workbook_2018-08-06_Contractor.xlsb,MTN.CRHOMG.COM,HL9YYZ1,\\MTNDCUT01,aaron.mabee,Oldcastle,19965 - 19997,NaN,NaN,-8342824394144019552
221,$U$25,22275,2018-08-13 03:38:46,QC(1),C:\Users\aaron.mabee\Dropbox\2018 Jobs and Pro...,Kuna to Meridian\Production\Contractor,QASP v1.1 workbook_2018-08-06_Contractor.xlsb,MTN.CRHOMG.COM,HL9YYZ1,\\MTNDCUT01,aaron.mabee,Oldcastle,19965 - 19997,NaN,NaN,1365780241677252576
227,$AG$30,22275,2018-08-13 03:39:11,QC(1),C:\Users\aaron.mabee\Dropbox\2018 Jobs and Pro...,Kuna to Meridian\Production\Contractor,QASP v1.1 workbook_2018-08-06_Contractor.xlsb,MTN.CRHOMG.COM,HL9YYZ1,\\MTNDCUT01,aaron.mabee,Oldcastle,19965 - 19997,NaN,NaN,-6843695537615975911
231,$AG$47,22275,2018-08-13 03:39:34,QC(1),C:\Users\aaron.mabee\Dropbox\2018 Jobs and Pro...,Kuna to Meridian\Production\Contractor,QASP v1.1 workbook_2018-08-06_Contractor.xlsb,MTN.CRHOMG.COM,HL9YYZ1,\\MTNDCUT01,aaron.mabee,Oldcastle,19965 - 19997,NaN,NaN,-4377203915106108201


In [4]:
# Getting the repeated one only
repeated = pd.read_excel("Result_cont_19965.xlsx", sheetname="Sheet1")
repeated = repeated.drop(["Var_time_gap", "Var_effect_type"], axis=1)
repeated["hash"] = repeated.Cell.str.cat(repeated["Sample"])
repeated['hash'] = repeated["hash"].apply(lambda x: hash(x))

In [5]:
# non_repeated_hash = list(set(all_data.hash).difference(set(repeated.hash)))
repeated_hash = set(repeated.hash)
non_repeated_data = []
for index, row in all_data.iterrows():
    if row["hash"] not in repeated_hash:
#         The repeated dataset doesn't have this value so we will just add it to repeated datacell as non repeat
        single_non_repeated_data = [row[3], row[0], row[1], row[2], "non-repeated", row["hash"]]
        non_repeated_data.append(single_non_repeated_data)
        
non_repeated = pd.DataFrame(non_repeated_data, columns = repeated.columns.values)
final_df = pd.concat([repeated, non_repeated])
final_df.head()

,Sample,Cell,Value,Time,Var_error,hash
0,QC(14),$U$32,1380.4,2018-08-21 04:25:35,Typo,-1140684387631145866
1,QC(14),$U$32,0,2018-08-21 04:26:20,Typo,-1140684387631145866
2,QC(18),$U$32,0,2018-08-22 03:43:38,Typo,9180307189236639823
3,QC(18),$U$32,0,2018-08-22 05:29:57,Typo,9180307189236639823
4,QC(18),$U$33,1530.4,2018-08-22 05:30:23,Fraud,7174634771668010374


In [6]:
final_df.to_csv("Cont_Typo_Fraud_NonRepeat_19965.csv",index=True,header = True)

In [7]:
data_Range = pd.read_excel("Cont_value_Range_19965.xlsx", sheetname='Sheet1')
data_Range.head()

,Var1_1,Var1_2,Var1_3
0,$U$32,0.0,0.0
1,$U$33,1530.4,1708.6
2,$U$37,2253.4,2393.4
3,$U$38,1308.0,1381.0
4,$Z$32,0.0,0.0


In [8]:
first_typo_first_fraud = pd.DataFrame(columns=set(final_df.Cell), index=set(final_df.Sample))
first_typo_last_fraud = pd.DataFrame(columns=set(final_df.Cell), index=set(final_df.Sample))
last_typo_first_fraud = pd.DataFrame(columns=set(final_df.Cell), index=set(final_df.Sample))
last_typo_last_fraud = pd.DataFrame(columns=set(final_df.Cell), index=set(final_df.Sample))
already_inserted = []
for index, row in final_df.iterrows():
    if row["hash"] in already_inserted:
        continue
    if row["Var_error"] == "non-repeated":
        first_typo_first_fraud[row["Cell"]][row["Sample"]] = float(row["Value"])
        first_typo_last_fraud[row["Cell"]][row["Sample"]] = float(row["Value"])
        last_typo_first_fraud[row["Cell"]][row["Sample"]] = float(row["Value"])
        last_typo_last_fraud[row["Cell"]][row["Sample"]] = float(row["Value"])
    elif row["Var_error"] == "Typo":
        # We will take the last one
        # Get all typo's entry of this test
        all_typo = final_df[final_df["hash"] == row["hash"]]
        all_typo["Value"] = all_typo["Value"].apply(pd.to_numeric, errors='coerce')
        all_typo = all_typo.dropna(subset=['Value'])
        all_typo.sort_values("Time", axis = 0, ascending = True, 
                 inplace = True, na_position ='last')
                # all_typo
        for i in range(all_typo.shape[0]):
            if any(all_typo.iloc[i]["Cell"] == data_Range['Var1_1']) and all_typo.iloc[i]["Value"] >= (data_Range[all_typo.iloc[i]["Cell"] == data_Range['Var1_1']]['Var1_2']).values[0] and all_typo.iloc[i]["Value"] <= (data_Range[all_typo.iloc[i]["Cell"] == data_Range['Var1_1']]['Var1_3']).values[0]:
                first_typo_first_fraud[row["Cell"]][row["Sample"]] = float(all_typo.iloc[i]["Value"])
                first_typo_last_fraud[row["Cell"]][row["Sample"]] = float(all_typo.iloc[i]["Value"])
                break
        for i in reversed(range(all_typo.shape[0])):
            if any(all_typo.iloc[i]["Cell"] == data_Range['Var1_1']) and all_typo.iloc[i]["Value"] >= (data_Range[all_typo.iloc[i]["Cell"] == data_Range['Var1_1']]['Var1_2']).values[0] and all_typo.iloc[i]["Value"] <= (data_Range[all_typo.iloc[i]["Cell"] == data_Range['Var1_1']]['Var1_3']).values[0]:
                last_typo_first_fraud[row["Cell"]][row["Sample"]] = float(all_typo.iloc[i]["Value"])
                last_typo_last_fraud[row["Cell"]][row["Sample"]] = float(all_typo.iloc[i]["Value"])
                break
#         else:
#             last_typo_first_fraud[row["Cell"]][row["Sample"]] = float(all_typo.iloc[-2]["Value"])
#             last_typo_last_fraud[row["Cell"]][row["Sample"]] = float(all_typo.iloc[-2]["Value"])
            
    elif row["Var_error"] == "Fraud":
        all_fraud = final_df[final_df["hash"] == row["hash"]]
        all_fraud["Value"] = all_fraud["Value"].apply(pd.to_numeric, errors='coerce')
        all_fraud = all_fraud.dropna(subset=['Value'])
        all_fraud.sort_values("Time", axis = 0, ascending = True, 
                 inplace = True, na_position ='last')
#         if all_fraud.shape[0] == 0:
#             continue
        for i in range(all_fraud.shape[0]):
            if any(all_fraud.iloc[i]["Cell"] == data_Range['Var1_1']) and all_fraud.iloc[i]["Value"] >= (data_Range[all_fraud.iloc[i]["Cell"] == data_Range['Var1_1']]['Var1_2']).values[0] and all_fraud.iloc[i]["Value"] <= (data_Range[all_fraud.iloc[i]["Cell"] == data_Range['Var1_1']]['Var1_3']).values[0]:
                first_typo_first_fraud[row["Cell"]][row["Sample"]] = float(all_fraud.iloc[i]["Value"])
                last_typo_first_fraud[row["Cell"]][row["Sample"]] = float(all_fraud.iloc[i]["Value"])
                break
        for i in reversed(range(all_fraud.shape[0])):
            if any(all_fraud.iloc[i]["Cell"] == data_Range['Var1_1']) and all_fraud.iloc[i]["Value"] >= (data_Range[all_fraud.iloc[i]["Cell"] == data_Range['Var1_1']]['Var1_2']).values[0] and all_fraud.iloc[i]["Value"] <= (data_Range[all_fraud.iloc[i]["Cell"] == data_Range['Var1_1']]['Var1_3']).values[0]:
                last_typo_last_fraud[row["Cell"]][row["Sample"]] = float(all_fraud.iloc[i]["Value"])
                first_typo_last_fraud[row["Cell"]][row["Sample"]] = float(all_fraud.iloc[i]["Value"])
                break
    else:
        print("Print Nothing")
        
    already_inserted.append(row["hash"])


In [9]:
first_typo_first_fraud = first_typo_first_fraud.astype(float)
first_typo_last_fraud = first_typo_last_fraud.astype(float)
last_typo_first_fraud = last_typo_first_fraud.astype(float)
last_typo_last_fraud = last_typo_last_fraud.astype(float)


last_typo_first_fraud.to_csv("last_typo_first_fraud_cont_19965.csv")
last_typo_last_fraud.to_csv("last_typo_last_fraud_cont_19965.csv")